In [9]:
# read csv into df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
from glob import glob
from datetime import datetime


print(DATA_GLOB_PLANINSKE_MAKRO_2022)

for path in glob(DATA_GLOB_PLANINSKE_MAKRO_2022):
    print(path)
    df_planine = pd.read_csv(path, sep=',', encoding='UTF-8')
    assert df_planine.columns[0] == 'datum', f"columns[0] = {df_planine.columns[0]}"
    assert df_planine.columns[1] == 'vhodi', f"columns[1] = {df_planine.columns[1]}"
    assert df_planine.columns[2] == 'izhodi', f"columns[2] = {df_planine.columns[2]}"
    print(df_planine.head())

    
    # split "datum" by space
    df_planine['datum'] = df_planine['datum'].str.split(' ')
    df_planine['datum_dan'] = df_planine['datum'].str[0]
    # df_planine['datum_ura'] = df_planine['datum'].str[1]
    # drop "datum" column
    df_planine = df_planine.drop(columns=['datum'])
    print(df_planine.head())
    # sum vhodi and izhodi based on "datum_dan"
    df_planine = df_planine.groupby('datum_dan').sum()
    print(df_planine.head())

    
    break


../data/planinstvo/2022/*.csv
              datum  vhodi  izhodi
0  2022-06-20 10:30      0       0
1  2022-06-20 11:00      0       0
2  2022-06-20 11:30      0       0
3  2022-06-20 12:00      2       0
4  2022-06-20 12:30      0       0
   vhodi  izhodi   datum_dan
0      0       0  2022-06-20
1      0       0  2022-06-20
2      0       0  2022-06-20
3      2       0  2022-06-20
4      0       0  2022-06-20
            vhodi  izhodi
datum_dan                
2022-06-20      3       0
2022-06-21      2       2
2022-06-22      0       0
2022-06-23      0       0
2022-06-24      1       0
